<a href="https://colab.research.google.com/github/10100111/Display-of-HW1/blob/main/75_Ultra_Lite_Pytorch_%D0%A7%D0%B0%D1%81%D1%82%D1%8C_1%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Задание Ultra Lite

Макс 10 баллов

Проведите эксперименты по классификации MNIST для различных значений гиперпараметров:

learningRate : [0.01,0.001]

размер скрытого слоя  : [128,256,512]

batchSizes : [32,64,128]

Результаты занесите в таблицу

In [ ]:
# Этим блоком будем визуализировать
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
import scipy
import pandas as pd
# Библиотеки Pytorch
import torch 
import torchvision # здесь лежат mnist, cifar и много других датасетов и трансформаций для картинок
import torch.nn as nn # здесь лежат все слои
import torch.utils.data as data # работа с загрузчиком данных
import torchvision.transforms as transforms # библиотека для преобразования изображений
import torchvision.datasets as dsets # работа с классом Датасет

# Вывод информации о модели
from torchsummary import summary

# Визуализации графа
from torch.utils.tensorboard import SummaryWriter 
import datetime, os
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Исследование 

# Функции


In [ ]:
# Cоздадим функцию генерации датасета
def getDataset(batchSize):
  # Скачиваем готовые данные и создаем датасет
  trainDataset = dsets.CIFAR10(root='./data/',                   # путь, где лежат данные
                              train=True,                       # создание данных из тренировочной выборки
                              transform=transforms.ToTensor(),  # здесь функция, превращающая PIL картинку в тензор
                              download=True)                    # True - данные скачиваются и помещаются в указанный путь

  # Загрузим MNIST
  batchSize = batchSize

  # Обучающая выборка
  trainDataset = dsets.MNIST(root='./data',
                            train=True,
                            transform=transforms.ToTensor(),
                            download=True)

  # Тестовая выборка
  testDataset = dsets.MNIST(root='./data', 
                            train=False, 
                            transform=transforms.ToTensor())

  # Dataset Loader (подготовка данных для сети)
  trainLoader = torch.utils.data.DataLoader(dataset=trainDataset, # Какой датасет
                                            batch_size=batchSize, # На сколько batch разделен
                                            shuffle=True) 

  testLoader = torch.utils.data.DataLoader(dataset=testDataset, #  Какой датасет
                                          batch_size=batchSize, # На сколько batch разделен
                                          shuffle=False)
  # Посмотрим на нашу выборку
  dataIter = iter(trainLoader) # По какой выборке пройдемся
  trainX, trainY = next(dataIter) # Присваиваем текущий batch

  return trainX, trainY

In [ ]:
# Создадим функцию генерации модели + summary
def getModel(input_size, hidden_size, num_classes):
  # Создадим переменную, в которой будет храниться тип устройства (GPU или CPU)
  device = 'cuda' if torch.cuda.is_available() else 'cpu'
  # Создаем класс
  class Classification(nn.Module):
      def __init__(self, input_size, hidden_size, num_classes):
          super().__init__() # наследуем все свойства + добавляем свое
          self.layer1 = nn.Linear(inputSize, hidden_size) # первый слой - линейный
          self.relu = nn.ReLU() 
          self.layer2 = nn.Linear(hidden_size, num_classes)
          self.softmax = nn.Softmax(dim=1) # Поскольку задача классификации, то используется функция активации softmax

      def forward(self, x): # Здесь мы прописываем принципы, по которым данные будут проходить через сеть
          out = self.layer1(x)  # выход первого слоя
          out = self.relu(out) # применяем функцию активации к выходу первого слоя
          out = self.layer2(out) # передаем это во второй слой
          out = self.softmax(out) # применяем функцию активации ко второму слою
          return out

  model = Classification(inputSize, hidden_size, numClasses).to(device) # Создаем объект нашей полносвязной сети
  summary(model, (inputSize,))
  return model

In [ ]:
def optimizer_and_loss(learningRate):  
  criterion = nn.CrossEntropyLoss() # функция ошибки
  optimizer = torch.optim.SGD(model.parameters(), lr=learningRate) # оптимизатор
  losses = [] # сюда будет добавляться средняя ошибка. Список будем использовать для построения графика
  model.train() # переход в режим обучения
  return criterion, optimizer

In [ ]:
# Создадим функцию обучения + вывод данных lr, hidden,  batchSize 
def trainModel(numEpochs):

  for epoch in range(numEpochs): # количество эпох
      lossTot = 0 # потери в совокупности

      for i, (images, labels) in enumerate(trainLoader): # проходимся по всем данным в batch
          images = images.view(-1, 28*28).to(device)     # приводим к правильному формату для сетки
          labels = labels.to(device)                    # отправляем метки на используемое устройство
          optimizer.zero_grad()                          # обнуляем градиент
          outputs = model(images)                        # здесь наше предсказание
          loss = criterion(outputs.log(), labels)        # считаем ошибку 
          loss.backward()                                # обратное распространение ошибки 
                                                        # x.grad += dloss/dx для всех параметров x

          lossTot += loss.cpu().detach().data # инкремент ошибки
          
          optimizer.step() # следующий шаг спуска

      losses.append(lossTot/len(trainDataset)) # вычисляем среднюю ошибку и добавляем в список
      print('Эпоха: [%d/%d], Ошибка: %.4f' 
            % (epoch+1, numEpochs, loss))
  # plt.plot(losses) # График нашего обучения

# Обучение

In [ ]:
inputSize = 784      # Размер входных данных
numClasses = 10      # Количество классов
numEpochs = 10       # Количество эпох
learningRate = [0.01, 0.001]     # Скорость обучения
hidden_size = [128, 256, 512]         # Размер скрытого слоя
batchSize = [32, 64, 128]        # Размер batchSize


In [ ]:
for lr in (learningRate):
  for h in (hidden_size):
    for bs in (batchSize):
      getDataset(batchSize = bs)
      getModel(input_size = inputSize, hidden_size = h, num_classes = numClasses)
      optimizer_and_loss(learningRate = lr)
      trainModel(numEpochs)
      print('lr = ', lr, 'hidden space =', h, 'batchSize = ', bs)

Files already downloaded and verified
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 128]         100,480
              ReLU-2                  [-1, 128]               0
            Linear-3                   [-1, 10]           1,290
           Softmax-4                   [-1, 10]               0
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.39
Estimated Total Size (MB): 0.39
----------------------------------------------------------------
Эпоха: [1/10], Ошибка: 0.1221
Эпоха: [2/10], Ошибка: 0.1030
Эпоха: [3/10], Ошибка: 0.0669
Эпоха: [4/10], Ошибка: 0.1919
Эпоха: [5/10], Ошибка: 0.1652
Эпоха: [6/10], Ошибка: 0.2748
Эпоха: [7/10], Ошибка: 0.0874
Эпоха: [8/10], Ошибка: 0.2925
Эпоха: [9/10], Ошибк

In [ ]:
df = pd.DataFrame({'Сочетание гиперпараметров':['lr =  0.001 hidden space = 512 batchSize =  128', 'lr =  0.001 hidden space = 512 batchSize =  64', 'lr =  0.001 hidden space = 512 batchSize =  32', 'lr =  0.001 hidden space = 256 batchSize =  128', 'lr =  0.001 hidden space = 256 batchSize =  64', 'lr =  0.001 hidden space = 256 batchSize =  32', 'lr =  0.001 hidden space = 128 batchSize =  128', 'lr =  0.001 hidden space = 128 batchSize =  64', 'lr =  0.001 hidden space = 128 batchSize =  32', 'lr =  0.01 hidden space = 512 batchSize =  128', 'lr =  0.01 hidden space = 512 batchSize =  64', 'lr =  0.01 hidden space = 512 batchSize =  32', 'lr =  0.01 hidden space = 256 batchSize =  128', 'lr =  0.01 hidden space = 256 batchSize =  64', 'lr =  0.01 hidden space = 256 batchSize =  32', 'lr =  0.01 hidden space = 128 batchSize =  128', 'lr =  0.01 hidden space = 128 batchSize =  64', 'lr =  0.01 hidden space = 128 batchSize =  32'], 'Значение ошибки': [0.1484, 0.0810, 0.1149, 0.2621, 0.2526, 0.1142, 0.2651, 0.0904, 0.1687, 0.1318, 0.2404, 0.2070, 0.0717, 0.1328, 0.2223, 0.1806, 0.1987, 0.3882]})

In [ ]:
df.head(5)

,Сочетание гиперпараметров,Значение ошибки
0,lr = 0.001 hidden space = 512 batchSize = 128,0.1484
1,lr = 0.001 hidden space = 512 batchSize = 64,0.0810
2,lr = 0.001 hidden space = 512 batchSize = 32,0.1149
3,lr = 0.001 hidden space = 256 batchSize = 128,0.2621
4,lr = 0.001 hidden space = 256 batchSize = 64,0.2526


In [ ]:
df.describe()

,Значение ошибки
count,18.000000
mean,0.181717
std,0.081481
min,0.071700
25%,0.119125
50%,0.174650
75%,0.235875
max,0.388200


In [ ]:
mask = df['Значение ошибки'].min()
df.loc[(df['Значение ошибки'] == mask)]

,Сочетание гиперпараметров,Значение ошибки
12,lr = 0.01 hidden space = 256 batchSize = 128,0.0717


# Вывод

Лучший результат ошибки = 0,0717. Это сочетание lr = 0.01, hidden = 256, batchSize = 128. 






# **Глоссарий**
*   **train dataset** - объект, включающий в себя ваш набор данных. На выходе такого объекта всегда должен быть тензор.
*   **train loader** - объект, в который вы подгружаете ваш train dataset, и который используете при обучении нейронной сети для итерирования по всему набору данных. 
*   **optimizer.zero_grad()** - обнуляет результат дифференциирования.
*   **loss.backward()** - считает градиент для обратного распространения ошибки. 
*   **optimizer.step()** - принимает (автоматически) посчитанные градиенты и делает шаг оптимизации.
*   **nn.Module** - базовый класс нейронной сети, на котором строится собственная нейронная сеть.
*   **def forward(self,x)** - функция-движитель. Связывает инициализированные слои. На вход принимается х (batch) и передается дальше по всем слоям.
*   **data.Dataset** -  базовый класс дата сета, на котором строится собственный датасет.
*   **.view()** - функция, интерпретирующая размер объекта для данной итерации в том виде, который указан в качестве параметра.
*   **.data** - получить исключительно статичные данные из тензора.
*   **.detach()** - открепить тензор от вычислительного графа.
*   **model.train()** - перевести объект нейронки в обучающи режим.
*   **model.eval()** - перевести объект нейронки в режим оценки.

# **Обратите внимание**

*   В параметрах тензора вы можете вручную прописать поддержку дифференциирования
*   Для удобства всегда создавайте DataLoader. Для DataLoader необходим Dataset
*   Dataset всегда можно прописать "вручную"
*   Прописываете архитектуру модели через отдельный класс
*   Выбирая функцию ошибки, смотрите, какой формат должен подаваться на вход
*   Во время обучения не забывайте о последовательности .zero_grad -> .backward -> .step
*   Режимы тренировочный и оценочный являются *разными* для Pytorch. Не забывайте указывать это в начале.    


